In [1]:
# Exercises:
# E01: train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?
# E02: split up the dataset randomly into 80% train set, 10% dev set, 10% test set. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see?
# E03: use the dev set to tune the strength of smoothing (or regularization) for the trigram model - i.e. try many possibilities and see which one works best based on the dev set loss. What patterns can you see in the train and dev set loss as you tune this strength? Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?
# E04: we saw that our 1-hot vectors merely select a row of W, so producing these vectors explicitly feels wasteful. Can you delete our use of F.one_hot in favor of simply indexing into rows of W?
# E05: look up and use F.cross_entropy instead. You should achieve the same result. Can you think of why we'd prefer to use F.cross_entropy instead?
# E06: meta-exercise! Think of a fun/interesting exercise and complete it.

## E01

In [1]:
# loading the data
import torch
import matplotlib.pyplot as plt
%matplotlib inline
import torch.nn.functional as F


words = open("names.txt", 'r').read().splitlines()

# words des
print("length : ",len(words))
print("max word length : ",max(len(w) for w in words))
print("min word length : ",min(len(w) for w in words))
print(words[:5])

length :  32033
max word length :  15
min word length :  2
['emma', 'olivia', 'ava', 'isabella', 'sophia']


In [2]:
# preprocessing
N = torch.zeros((27,702), dtype=torch.int32)

a = '.abcdefghijklmnopqrstuvwxyz'
d_stoi = {}
count = 0
for i in a:
    for j in a[1:]:
        d_stoi[(i+j)] = count
        count += 1

s_stoi = {}
count = 0
for i in a:
    s_stoi[i] = count
    count += 1
s_stoi

# reverse stoi
d_itos = {i:s for s,i in d_stoi.items()}
s_itos = {i:s for s,i in s_stoi.items()}


for w in words:
    w = '.' + w + '.'
    length = len(w)
    if length > 1:
        for i in range(length - 1): 
            try: 
                ix1 = d_stoi[(w[i:i+2])]
                ix2 = s_stoi[(w[i+2])]
            except:
                continue
            N[ix2, ix1] += 1

In [3]:
out = []
P = (N+1).float()
row_sum = P.sum(1, keepdim=True) 

P /= row_sum
P[0].sum()

tensor(1.)

In [4]:
# loss
log_likelihood = 0.0
counter = 0

for w in words:
    w = '.' + w + '.'
    length = len(w)
    if length > 1:
        for i in range(length - 1): 
            try: 
                ix1 = d_stoi[(w[i:i+2])]
                ix2 = s_stoi[(w[i+2])]
            except:
                continue
            N[ix2, ix1] += 1
            prob = P[ix2, ix1]
            logprob = torch.log(prob)
            log_likelihood += logprob
            counter +=1



negative_log_likelihood = -log_likelihood
avg_negative_log_likelihood = negative_log_likelihood / counter

print(f'{log_likelihood=}')
print(f'{negative_log_likelihood=}')
print(f'{avg_negative_log_likelihood=}')

log_likelihood=tensor(-919222.3750)
negative_log_likelihood=tensor(919222.3750)
avg_negative_log_likelihood=tensor(4.6872)


In [23]:
# sampling
g = None
out = []
for i in range(5):
    name =''
    is_start = True
    while True:
        if is_start:
            ix = torch.multinomial(P[0][:26], num_samples=1, replacement=True,generator = g).item()
            name +=d_itos[ix]
            ix = torch.multinomial(P[:,ix], num_samples=1, replacement=True,generator = g).item()
            name += s_itos[ix]
            is_start = False
        else:
            to_fed = name[-2:] # last two string
            ix = d_stoi[to_fed]
            ix = torch.multinomial(P[:,ix], num_samples=1, replacement=True,generator = g).item()
            name += s_itos[ix]
        if ix == 0 or len(name) > 15:  #if its find '.'
            if len(name) > 15: name += '.'
            break

    out.append(name[1:-1])
print(out)

['phjydance', 'vivivmylobettcx', 'copqkxvycmvwqjg', 'lukwbmsjcqsuwwq', 'kipdtcxtiqgpqxw']


## E02